<a href="https://colab.research.google.com/github/TBab0/duth_cv4/blob/main/2tuned%20vertical_flip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Declare Imports
from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils, to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
import os
from google.colab import drive
import zipfile
import tensorflow as tf
import datetime

In [2]:
# Import from drive
drive.mount('/content/drive')
local_zip = '/content/drive/MyDrive/imagedb_btsd.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

train_dir = '/content/imagedb'
test_dir = '/content/imagedb_test'

Mounted at /content/drive


In [3]:
# Prep the Train Valid and Test directories for the generator
batch_size = 128
row, col, cha = 256, 256, 3
target_size = (row, col)

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   vertical_flip= True,
                                   validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training', seed=1) # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',  seed=1) # set as validation data

Found 2457 images belonging to 34 classes.
Found 599 images belonging to 34 classes.


In [4]:
# Convoluted Base MODEL
conv_base = ResNet50(weights='imagenet', include_top=False, input_shape=(row, col, cha))
# print(conv_base.summary())

# Freeze the layers
for layer in conv_base.layers[:]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
# for layer in conv_base.layers:
    # print(layer, layer.trainable)

94765736/94765736 [==============================] - 3s 0us/step


In [5]:
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(conv_base)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(2048, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(34, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                                 
 dense (Dense)               (None, 2048)              268437504 
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 34)                1

In [6]:
callbacks = []

save_best_callback = tf.keras.callbacks.ModelCheckpoint(f'best_weights.hdf5', save_best_only=True, verbose=1)
callbacks.append(save_best_callback)

early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, verbose=1)
callbacks.append(early_stop_callback)


In [7]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizers.adam_v2.Adam(learning_rate=1e-4), metrics=['acc'])
# Train the model
history = model.fit_generator(train_generator, steps_per_epoch=train_generator.samples/train_generator.batch_size, epochs=30, validation_data=validation_generator, verbose=1, callbacks = callbacks)
 
# Save the model
model.save('small_last4.h5')

<ipython-input-7-24e1b0ab929e>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, steps_per_epoch=train_generator.samples/train_generator.batch_size, epochs=30, validation_data=validation_generator, verbose=1, callbacks = callbacks)


Epoch 1/30
20/19 [===============================] - ETA: 0s - loss: 4.9630 - acc: 0.4155
Epoch 1: val_loss improved from inf to 0.84083, saving model to best_weights.hdf5
19/19 [==============================] - 46s 2s/step - loss: 4.9630 - acc: 0.4155 - val_loss: 0.8408 - val_acc: 0.7646
Epoch 2/30
20/19 [===============================] - ETA: 0s - loss: 1.7669 - acc: 0.6915
Epoch 2: val_loss improved from 0.84083 to 0.49917, saving model to best_weights.hdf5
19/19 [==============================] - 30s 2s/step - loss: 1.7669 - acc: 0.6915 - val_loss: 0.4992 - val_acc: 0.8681
Epoch 3/30
20/19 [===============================] - ETA: 0s - loss: 1.0346 - acc: 0.7928
Epoch 3: val_loss improved from 0.49917 to 0.36799, saving model to best_weights.hdf5
19/19 [==============================] - 29s 2s/step - loss: 1.0346 - acc: 0.7928 - val_loss: 0.3680 - val_acc: 0.9048
Epoch 4/30
20/19 [===============================] - ETA: 0s - loss: 0.7069 - acc: 0.8405
Epoch 4: val_loss improved fr

In [8]:
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
test_datagen  = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator =  test_datagen.flow_from_directory(test_dir,
                                                        batch_size=128,
                                                        class_mode='categorical',
                                                        #  color_mode='grayscale',
                                                         target_size=target_size) 
loss, acc = model.evaluate(test_generator)

Found 2149 images belonging to 34 classes.
17/17 [==============================] - 11s 627ms/step - loss: 0.3416 - acc: 0.9195
